# requirements

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, precision_score
import joblib

# loading data and preprocessing

In [ ]:
file_path = 'data/premier-league-matches.csv'  # Update this path as needed
matches = pd.read_csv(file_path)

matches["date"] = pd.to_datetime(matches["Date"])
matches["h/a"] = matches["Home"].astype("category").cat.codes
matches["opp"] = matches["Away"].astype("category").cat.codes
matches["hour"] = 0  # Assuming time is not provided, set a default value
matches["day"] = matches["date"].dt.dayofweek
matches["target"] = (matches["FTR"] == "H").astype("int")

# building the model

In [ ]:
# train-test split
train = matches[matches["date"] < '2022-01-01']
test = matches[matches["date"] >= '2022-01-01']
predictors = ["h/a", "opp", "hour", "day"]
